In [ ]:
!git clone https://github.com/facebookresearch/detectron2.git
!cd detectron2 && python -m pip install -e .

Cloning into 'detectron2'...
remote: Enumerating objects: 15054, done.
remote: Counting objects: 100% (79/79), done.
remote: Compressing objects: 100% (44/44), done.
remote: Total 15054 (delta 37), reused 61 (delta 35), pack-reused 14975
Receiving objects: 100% (15054/15054), 6.12 MiB | 6.93 MiB/s, done.
Resolving deltas: 100% (10902/10902), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/detectron2
  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB

In [ ]:
pip install cython; pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/cocodataset/cocoapi.git to /tmp/pip-req-build-wki9d_s9
  Running command git clone --filter=blob:none --quiet https://github.com/cocodataset/cocoapi.git /tmp/pip-req-build-wki9d_s9
  Resolved https://github.com/cocodataset/cocoapi.git to commit 8c9bcc3cf640524c4c20a9c40e89cb6a2f2fa0e9
  Preparing metadata (setup.py) ... done
  Created wheel for pycocotools: filename=pycocotools-2.0-cp310-cp310-linux_x86_64.whl size=386200 sha256=bcfaa1efba0bc5fd53ae72e7ce637436dd4ea1a3cc37028104d4a95745ecb470
  Stored in directory: /tmp/pip-ephem-wheel-cache-o75dx_je/wheels/39/61/b4/480fbddb4d3d6bc34083e7397bc6f5d1381f79acc68e9f3511
Successfully built pycocotools
  Attempting uninstall: pycocotools
    Found existing installation: pycocotools 2.0.6
    Uninstalling pycocotools-2.

In [ ]:
!pip install detectron2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!wget https://dl.fbaipublicfiles.com/detectron2/COCO-Detection/faster_rcnn_R_101_FPN_3x/137851257/model_final_f6e8b1.pkl


--2023-05-25 14:01:56--  https://dl.fbaipublicfiles.com/detectron2/COCO-Detection/faster_rcnn_R_101_FPN_3x/137851257/model_final_f6e8b1.pkl
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 65.8.11.91, 65.8.11.49, 65.8.11.16, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|65.8.11.91|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 243480188 (232M) [application/octet-stream]
Saving to: ‘model_final_f6e8b1.pkl’

model_final_f6e8b1. 100%[===================>] 232.20M  56.1MB/s    in 4.6s    

2023-05-25 14:02:02 (50.7 MB/s) - ‘model_final_f6e8b1.pkl’ saved [243480188/243480188]



In [ ]:
#from detectron2.config import get_cfg
#from detectron2 import model_zoo
#from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog
import cv2

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 34  # 주의: 클래스 개수 변경


ModuleNotFoundError: ignored

In [ ]:
from detectron2.data import DatasetCatalog, MetadataCatalog

def get_car_dicts(img_dir):
    dataset_dicts = []
    # 주의: 이 부분은 실제 데이터셋에 따라 수정해야 합니다.
    for idx, v in enumerate(img_dir):
        record = {}
        filename = v["file_name"]
        height, width = cv2.imread(filename).shape[:2]

        record["file_name"] = filename
        record["image_id"] = idx
        record["height"] = height
        record["width"] = width

        objs = []
        for anno in v["annotations"]:
            # 주의: 이 부분은 실제 데이터셋에 따라 수정해야 합니다.
            obj = {
                "bbox": [anno["bbox"][1], anno["bbox"][0], anno["bbox"][3], anno["bbox"][2]],
                "bbox_mode": BoxMode.XYXY_ABS,
                "segmentation": [anno["segmentation"]],
                "category_id": anno["category_id"],
                "iscrowd": anno["iscrowd"]
            }
            objs.append(obj)
        record["annotations"] = objs
        dataset_dicts.append(record)
    return dataset_dicts

# 데이터셋을 등록합니다.
for d in ["train", "val"]:
    DatasetCatalog.register("car_" + d, lambda d=d: get_car_dicts("car/" + d))
    MetadataCatalog.get("car_" + d).set(thing_classes=["car"])
car_metadata = MetadataCatalog.get("car_train")
